In [ ]:
#Clustering
#Clustering.py는 문장 임베딩 유사성을 기반으로 유사한 문장을 클러스터링하는 예를 보여줍니다.
# 참고 소스 : https://github.com/BM-K/KoSentenceBERT-ETRI

from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
import pandas as pd
import time
from os import sys

sys.path.append('../')
from myutils import GPU_info, seed_everything, mlogging, bi_encoder

logger = mlogging(loggername="cluster", logfilename="../../../log/cluster")
device = GPU_info()
#-----------------------------------
# 파라메터
#-----------------------------------
#device = 'cpu'  # cpu 테스트 할때
max_num_clusters = 100  # 최대 클러스터링을 몇개로 할지
search_k = 5       # 라벨링(카테고리) 검색을 몇개까지 할지
seed = 111
#-----------------------------------
seed_everything(seed)

In [ ]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
import torch
from myutils import bi_encoder, dense_model, onnx_model, onnx_embed_text

bi_encoder_path = "bongsoo/kpf-sbert-128d-v1" #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls

 # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 
out_dimension = 128

word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
  
print(f'\n---bi_encoder---------------------------')
print(bi_encoder1)
print(word_embedding_model1)
#------------------------------------------------------------------------------------------------------------------------


In [ ]:
#-------------------------------------------------------------------------------------------------------
# 2. JSON 파일 로딩 후 df 만듬
# => KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# list 데이터들을 dataframe으로 만듬
'''
contexts = [
        [1, '독도 독도 독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다. 헬기 동체 잔해물과 부유물 등은 발견되고 있지만 정작 실종자들은 발견하지 못해 수색이 장기화될 것이라는 우려가 현실이 될 조짐을 보여 실종자 가족들의 애를 태우고 있다.범정부현장수습지원단(지원단)은 10일 오전 10시 브리핑에서 이날 오전까지 독도해역 수색 결과 4점의 부유물을 추가 발견, 인양했다고 밝혔다'], 
        [11,'대구시는 11일 내년 정부 예산 3조1330억원을 확보해 전년보다 2%(611억원) 늘어났다고 밝혔다.그러나 올해 국비 증액 규모(1817억원)와 비교해 절반 수준에 그치며, 물산업클러스터 r&d 사업(200억), 국립청소년진로직업체험 수련원 건립 등은 한푼도 반영되지 않았다.'],
        [21,'자유한국당 (이름) 의원이 "공수처는 정권 유지를 위한 수단으로, 공수처가 있었으면 조국 수사를 못했을 것"이라고 주장했다.최 의원은 18일 대구 호텔수성에서 열린 대구 경북중견언론인모임 <아시아포럼21> 토론회에서 "검찰은 정권 말기가 되면 정권에 칼을 들이댔다. 공수처는 검찰의 칼끝을 회피하기 위한 것"이라며 이같이 말했다'],
        [2,'27일 오전 천연기념물(201-2호)이며 멸종위기 2급인 큰고니 떼가 경북 포항시 북구 흥해읍 샛강에 날아들었다.이날 관측된 큰고니는 어미 8마리와 새끼 3마리다.앞서 지난 19일 큰고니 8마리가 올들어 처음으로 관측됐다.큰고니들은 포항시 흥해읍 샛강에서 수초 등을 먹고 휴식한 후 대구 안심습지와 낙동강 하구 등지로 날아갈 것으로 보인다'],
        [3,'3일 대구와 경북지역은 구름이 많겠으며, 동해안과 북부 내륙에는 비가 내릴 것으로 예상된다.대구기상청에 따르면 중국 북동지방에서 남하하는 고기압의 가장자리에 들어 구름이 많겠으며, 경북 북부 내륙은 북쪽을 지나는 약한 기압골의 영향을 받아 오후부터 비가 약하게 내리거나 빗방울이 떨어지겠다.'],
        [31,'각급 정부기관의 통신망을 관리할 국가정보자원관리원 대구센터가 31일 대구 동구 도학동에서 착공했다.4312억원을 투입해 2021년 8월 준공 예정인 대구센터는 86개 기관의 서비와 장비를 운용하게 된다.정부통합전산센터에서 명칭이 변경된 국가정보자원관리원은 현재 대전본원과 광주센터를 운영 중이다.대구센터는 급변하는 행정환경과 4차 산업혁명 시대에 맞춰 클라우드, 빅데이터 등 신기술이 접목된 지능형 전산센터로 구축된다'],
        [4,'26일 오후 11시44분쯤 대구 달서구 신당동의 한 아파트에서 원인 모를 불이 나 주민 1명이 다치고 수십명이 대피했다.27일 소방당국에 따르면 신고를 받고 소방차 24대와 소방대원 74명이 출동해 20분만에 불길을 잡았다.이 불로 아파트 2층에 사는 a씨(53)가 온 몸에 화상을 입어 병원으로 이송됐으며, 연기에 놀란 주민 50여명이 대피하는 소동을 빚었다.']
       ]
df_contexts = pd.DataFrame(contexts, columns=['contextid','context'])
print(df_contexts['context'].values)
'''

# KorQuAD_v1.0 혹은 aihub 뉴스 기사 기계독해 데이터 QuA 파일을 불러옴.
jsonfile = '../embedding_sample/faiss/data/KorQuAD_v1.0_dev.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_korquad_v1_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 3. 문단(문서)들을 문장으로 분리
# =>df_contexts에 대해 각 문장별 '.'(마침표) 로 구분해서 sub 문장을 만듬. 혹은 kss 이용해서 sub 문장을 만듬
#-------------------------------------------------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import split_sentences

#text = ["해외에서 데이터 무제한?? 뿐 아니라 음성과 문자 요금을 할인 받을 수 있는 ++요금제가 나왔다. 날씨는 좋다. lg유플러스는 중국, 일본, 홍콩, 싱가포르, 필리핀 등 아시아 8국을 대상으로~~ 무제한 데이터와 음성, 문자를 할인해주는 '스마트 로밍 요금제' 2종을 오는 28일부터 판매한다고 27일 밝혔다. . . 우선 '스마트 로밍음성'은 하루 기본료가 2000원으로 음성발신은 1분당 1000원이며, 문자메시지(sms)와 멀티미디어 문자메시지(mms)는 1건당 150원이다. 종전 500원에서 350원 인하했다. . . '스마트 로밍패키지'는 여기에 데이터 무제한 로밍서비스가 더해져 하루 기본료가 1만1000원이다. 기본료는 로밍 기간에 상관없이 사용한 당일에만 청구된다. . . 스마트 로밍 요금제는 일단 오는 7월 말까지 프로모션 형태로 제공하고 이후 정식 요금제로 추진한다는 계획이다. . . 아울러 해외에서 무제한 데이터 로밍 서비스와 데이터로밍 차단을 신청·해지할 수 있는 로밍 전용 모바일 홈페이지((이메일))도 운영한다. . . (이름) lg유플러스 글로벌로밍팀장은 '아시아 출(이름) 여행을 계획하고 있는 고객들이 저렴한 비용으로 안심하고 로밍 서비스를 사용할 수 있도록 이번 프로모션 요금제를 출시했다'며 '지속적으로 해외 로밍 이용 고객들의 편의성 증대를 위한상품을 준비할 예정'이라고 말했다. . . . . . . . # # #. . ■ 사진설명. . lg유플러스(부회장 (이름) / (이메일) )가 4월 28일부터 중국, 일본 등 아시아 8개국*을 대상으로 데이터와 음성 문자까지 할인해서 제공하는 스마트 로밍요금제(스마트 로밍음성/스마트 로밍패키지) 2종을 출시했다."]
#text =["오늘은 날씨가 ^^~~~~좋다. 내일은 비가 오고 춥겠다고 한다^^. 걱정이다. 오늘만큼만 매일 날씨가 좋으면 좋겠다.~~~~"]
sub_contexts = []
sentences = []

contexts = df_contexts['context'].values.tolist()
#contexts = text
start = time.time()
sub_contexts = split_sentences(paragraphs=contexts, sentences_split_num=10, showprogressbar=True, paragraphs_num=999, debug=False)
logger.info(f'*문장 분리 시간 : {time.time()-start:.4f}')

print(sub_contexts[0])

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 4. 문단에 문장들의 평균 임베딩을 구한다.
#-------------------------------------------------------------------------------------------------------
from myutils import embed_text

start = time.time()

#-------------------------------------------------------------
# 각 문단의 문장들에 벡터를 구하고 평균값을 리스트에 저장해 둠.
embeddings_list = []
for sentences in tqdm(sub_contexts):
    avg_paragraph_vecs = embed_text(model=bi_encoder1, paragraphs=sentences, return_tensor=False)
    arr = np.array(avg_paragraph_vecs)
    avg_paragraph_vec = arr.mean(axis=0)
    embeddings_list.append(avg_paragraph_vec.ravel(order='C')) # 1차원 배열로 변경후 리스트에 저장
    
#list를 numpy.array로 변경
embeddings_arr = np.array(embeddings_list).astype("float32")

print('shape: {}'.format(embeddings_arr.shape))
print('type: {}\n'.format(type(embeddings_arr)))
logger.info(f'*임베딩 시간 : {time.time()-start:.4f}')

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 4. 클러스터링 
# -> 실루엣 점수가 가장 높은 최적 클러스터 수를 계산 후 클러스터링 진행함.
#-------------------------------------------------------------------------------------------------------
# Then, we perform k-means clustering using sklearn:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#클러스터링 계수는 문단의 계수보다는 커야 함. 
assert max_num_clusters <= len(sub_contexts), f"max_num_clusters > len(sub_contexts)"

start = time.time()
#-------------------------------------------------------------

# 실루엣 점수가 가장 높은 최적 클러스터수를 계산해서 클러스터링함.
# 클러스터 범위 지정
nstart = int(max_num_clusters/10)
if nstart < 2:
    nstart = 2
  
n_clusters_range = range(nstart, max_num_clusters)
print(f'실루엣 스코어 계산을 위한 클러스터 범위 : {nstart}~{max_num_clusters}')

# 실루엣 점수 저장할 리스트
silhouette_scores = []

print(f'*실루엣 스코어(클러스터수 : 스코어)')
idx = 2
for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=seed)
    kmeans.fit(embeddings_arr)
    silhouette_scores.append(silhouette_score(embeddings_arr, kmeans.labels_))
    if idx % 10 == 0:
        print(f'{idx} : {silhouette_score(embeddings_arr, kmeans.labels_)}')
    idx+=1

# 최적의 클러스터 수 구함.
best_n_clusters = n_clusters_range[silhouette_scores.index(max(silhouette_scores))]

print(f'-------------------------------------------')
print(f'*Best 실루엣  클러스터 수: {best_n_clusters} / 스코어:{silhouette_scores[best_n_clusters-nstart]}')
print()
#-------------------------------------------------------------
# 클러스터링 진행

clustering_model = KMeans(n_clusters=best_n_clusters, init='k-means++', random_state=seed)
clustering_model.fit(embeddings_arr)
cluster_assignment = clustering_model.labels_

logger.info(f'*클러스터링 시간 : {time.time()-start:.4f}')
#-------------------------------------------------------------

clustered_sentences = [[] for i in range(best_n_clusters)]
clustered_list = [[] for i in range(best_n_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_list[cluster_id].append(sentence_id)
    clustered_sentences[cluster_id].append("["+str(sentence_id)+"] "+contexts[sentence_id])

for i, clusters in enumerate(clustered_sentences):
    if i < 2:
        print("Cluster ", i+1)
        for cluster in clusters:
            print(cluster)
            print("")

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 5. 라벨링(카테고리 폴더명) 이 있으면 라벨링명칭 임베딩 벡터를 구해서 Faiss에 추가.
#-------------------------------------------------------------------------------------------------------
from myutils import embed_text, fassi_index

label_list = ['건강,의료,병원', '경제,금융,부동산,증권', '교육,학교', '국제', '라이프스타일', 
              '사건,사고', '사회일반', '산업,운송,철강,해운', '스포츠,운동', '여성,복지', 
              '여행,레저,관광', '정치,국회', '지역,로컬', '취미,여가생활', 'IT,과학,인터넷',
              '역사,종교', '문화,예술,영화,음악', '세계', '노동,일,직업', '식품,음료,과일,생필품']

label_vectors = embed_text(model=bi_encoder1, paragraphs=label_list, return_tensor=False)
print('shape: {}'.format(label_vectors.shape))
print('type: {}\n'.format(type(label_vectors)))

index = fassi_index(embeddings=label_vectors, method=0)

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 6. 클러스터링 평균 벡터로 라벨링 Faiss k=5 검색.
#-------------------------------------------------------------------------------------------------------

#-----------------------------------------------------
# 클러스터링 문단들의 평균 벡터를 구함.
avg_vect_list = []

for i, clusters in enumerate(clustered_list):
    count = 0;
    vect = np.zeros((1,embeddings_arr[0].shape[0]))
   # print(embeddings_arr[0].shape[0])
    for sentence_id in clusters:
        vect += embeddings_arr[sentence_id]
        count += 1
        
    vect /= count
    avg_vect_list.append(vect.ravel(order='C'))
#-----------------------------------------------------   
    
print(f'avg_vect_len: {len(avg_vect_list)}')
print(f'avg_vect[0]_len: {len(avg_vect_list[0])}')

# Faiss 라벨링  k=5 검색.
# 입력은 2차원 배열이어야함 (예:(12,768)
distance, idx = index.search(np.array(avg_vect_list).astype("float32"), k=search_k) # 단어별 검색해서 5개 스코어를 얻어옴.

# bi_predictions_list 리스트에 distance(스코어)와 idx(라벨링 id) 을 쌍으로 저장해 둠
bi_predictions_list = []
for a, b in zip(distance, idx):
    pair = (a, b)
    bi_predictions_list.append(pair)
    
print(bi_predictions_list[0])
print(label_list[bi_predictions_list[0][1][0]])

In [ ]:
# 클러스터링 문단과 라벨링 값, 스코어등을 출력해봄.
for i, clusters in enumerate(clustered_sentences):
    
    if i >= 0 and i <= 5:
        print("Cluster ", i+1)
        
        # 라벨링 목록들 출력
        labelstr: str = '*라벨링: '
        for label in bi_predictions_list[i][1]:
            labelstr += label_list[label] + '/ '
        print(labelstr)
        
        # 스코어 출력
        print('*스코어:{}'.format(bi_predictions_list[i][0]))
        print()
        
        # 클러스터링 문단들 출력
        for cluster in clusters:
            print(cluster)
            print("")

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 7. 검색된 10 카테고리 폴더명과 문서를 cross-encoder로 스코어 구함.
#-------------------------------------------------------------------------------------------------------

from sentence_transformers.cross_encoder import CrossEncoder

start = time.time()
#-------------------------------------------------------------------------------------------------------
# cross-encoder 모델 로딩
#-------------------------------------------------------------------------------------------------------
cross_encoder_path = "bongsoo/klue-cross-encoder-v1"# klue-cross-encoder-v1 # albert-small-kor-cross-encoder-v1 # kpf-cross-encoder-v1
#cross_encoder_path = "../../../data11/model/moco/cross/kpf-cross-sts4"
cross_encoder = CrossEncoder(cross_encoder_path, max_length=512, num_labels=1, device=device)
#-------------------------------------------------------------------------------------------------------

cross_predictions_list = []

for i, clusters in enumerate(tqdm(clustered_list)):
    sentence_combinations = []
    label_idx_list = []
    for label in bi_predictions_list[i][1]:
        sentence_combinations.append([contexts[clusters[0]], label_list[label]]) # ['문장','라벨링'] 쌍으로 문장 만듬
        if i == 0:
            print(f'{label_list[label]}/{label}')
        label_idx_list.append(label) # 라벨링 번호를 저장해 둠.[17,3,8,0,11,13,5,15,16]
        

    # cross-enocoder 돌려서 출력된 score 추가함.
    cross_scores = cross_encoder.predict(sentence_combinations)+1
    if i == 0:
        print(f'\n*cross_scores:{cross_scores}\n')
    
    # 내림차순으로 정렬
    dec_cross_idx = reversed(np.argsort(cross_scores))
    tmp_list = []
    # 내림차순 정렬한 라벨과 라벨idx 출력 해봄.
    for idx in dec_cross_idx:
        if i == 0:
            print(f'idx:{idx}-predicts:{label_list[label_idx_list[idx]]}/{label_idx_list[idx]}')
        tmp_list.append(label_idx_list[idx])
      
    # 라벨링 idx 목록을 저장해 둠.
    cross_predictions_list.append(tmp_list) 
    
logger.info(f'*cross_encoder 시간 : {time.time()-start:.4f}')

print(f'\ncross_predictions_list[0]: {cross_predictions_list[0]}')   

In [ ]:
count = 1
for bi_pred, cross_pred in zip(bi_predictions_list, cross_predictions_list):
    print('{}: {}->{}'.format(count, bi_pred[1], cross_pred))
    count += 1